## Calculate pO2 for DPLE

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
var1 = 'O2' # mmol/m3
var2 = 'TEMP' # deg C
var3 = 'SALT' # PSS

### DPLE monthly ens-mean variables - T-bar, S, O2

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
# O2
filename = f'{path}/DPLE_driftcorrected_{var1}_mean_ens_mean.nc'
ds_dple = xr.open_dataset(filename)
O2 = ds_dple[f'{var1}_mean'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
# TEMP
filename = f'{path}/DPLE_driftcorrected_{var2}_mean_ens_mean.nc'
ds_dple = xr.open_dataset(filename)
TEMP = ds_dple[var2].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
# SALT
filename = f'{path}/DPLE_driftcorrected_{var3}_mean_ens_mean.nc'
ds_dple = xr.open_dataset(filename)
SALT = ds_dple[var3].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))

### Depth

In [5]:
depth = ds_dple['z_t']*TEMP/TEMP/1e2 # m

In [6]:
depth = depth.transpose("L", "z_t", "nlat", "nlon")

In [7]:
depth

<xarray.DataArray (L: 122, z_t: 35, nlat: 140, nlon: 195)>
array([[[[         nan,          nan,          nan, ...,   5.        ,
            5.        ,   5.        ],
         [         nan,          nan,          nan, ...,   5.        ,
            5.        ,   5.        ],
         [         nan,          nan,          nan, ...,   5.        ,
            5.        ,   5.        ],
         ...,
         [  5.        ,   5.        ,   5.        , ...,          nan,
                   nan,          nan],
         [  5.        ,   5.        ,   5.        , ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan]],

        [[         nan,          nan,          nan, ...,  15.        ,
           15.        ,  15.        ],
         [         nan,          nan,          nan, ...,  15.        ,
           15.        ,  15.        ],
         [         nan,          nan,          nan, ...,  15.        ,
           15.        ,  15.        ],
...
         [527.72800781, 527.72800781, 527.72800781, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan]],

        [[         nan,          nan,          nan, ..., 579.37289062,
          579.37289062, 579.37289062],
         [         nan,          nan,          nan, ..., 579.37289062,
          579.37289062, 579.37289062],
         [         nan,          nan,          nan, ..., 579.37289062,
          579.37289062, 579.37289062],
         ...,
         [579.37289062, 579.37289062, 579.37289062, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan]]]])
Coordinates:
  * z_t      (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon

In [8]:
%%time
pO2 = mi.compute_pO2(O2, TEMP, SALT, depth)

CPU times: user 39.2 s, sys: 14.3 s, total: 53.6 s
Wall time: 1min 41s


### save pO2

In [9]:
dsf = xr.Dataset()
dsf['pO2'] = pO2
dsf['3ddepth'] = depth

In [10]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, L: 122)
Coordinates:
  * z_t      (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    Y        int64 1954
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    pO2      (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    3ddepth  (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [11]:
%%time
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_pO2_mean_dimension_reduced_ens_mean_monthly.nc', mode='w')

CPU times: user 43.8 ms, sys: 657 ms, total: 701 ms
Wall time: 931 ms
